# Package Imports

In [1]:
import glob
import os
import numpy as np
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
sns.set(font_scale=1.2, context="paper", style="ticks")
sns.set_palette("viridis")

import mysql.connector as mariadb
from astropy.time import Time

%matplotlib inline

import plotly
plotly.offline.init_notebook_mode(connected=True)

import sys
sys.path.append("../..")

In [2]:
os.environ.get("CONDA_PREFIX")

'/epyc/opt/anaconda'

In [3]:
os.nice(1)

1

In [4]:
import thor

## Run THOR

In [5]:
columnMapping = {        
        
        # Observation ID
        "obs_id" : "obs_id",
        
        # Exposure time
        "exp_mjd" : "exp_mjd",
        
        # Visit ID
        "visit_id" : "visit_id",
        
        # Field ID
        "field_id" : "field",
        
        # Field RA in degrees
        "field_RA_deg" : "fieldRA_deg",
        
        # Field Dec in degrees
        "field_Dec_deg" : "fieldDec_deg",
        
        # Night number
        "night": "nid",
        
        # RA in degrees
        "RA_deg" : "ra",
        
        # Dec in degrees
        "Dec_deg" : "decl",
        
        # Observer's x coordinate in AU
        "obs_x_au" : "HEclObsy_X_au",
        
        # Observer's y coordinate in AU
        "obs_y_au" : "HEclObsy_Y_au",
        
        # Observer's z coordinate in AU
        "obs_z_au" : "HEclObsy_Z_au",
        
        # Magnitude (UNUSED)
        "mag" : "magpsf",
        
        ### Truth Parameters
        
        # Object name
        "name" : "designation",
        
        # Observer-object distance in AU
        "Delta_au" : "Delta_au",
        
        # Sun-object distance in AU (heliocentric distance)
        "r_au" : "r_au",
        
        # Object's x coordinate in AU
        "obj_x_au" : "HEclObj_X_au",
        
        # Object's y coordinate in AU
        "obj_y_au" : "HEclObj_Y_au",
        
        # Object's z coordinate in AU
        "obj_z_au" : "HEclObj_Z_au",
        
        # Object's x velocity in AU per day
        "obj_dx/dt_au_p_day" : "HEclObj_dX/dt_au_p_day",
        
        # Object's y velocity in AU per day
        "obj_dy/dt_au_p_day" : "HEclObj_dY/dt_au_p_day",
        
        # Object's z velocity in AU per day
        "obj_dz/dt_au_p_day" : "HEclObj_dZ/dt_au_p_day",
        
        # Semi-major axis
        "a_au" : "a_au",
        
        # Inclination
        "i_deg" : "i_deg",
        
        # Eccentricity
        "e" : "e",
    }

In [6]:
#simulated_ephemeris = pd.read_csv("../analysis/ztf/known_object_observations_nid_610_624.eph", sep=" ", index_col=False, low_memory=False)
simulated_ephemeris = thor.readEPHFile("../../analysis/ztf/MPCORB_20181106_ZTF.eph")
#orbits = pd.read_csv("../../analysis/ztf/known_object_observations_nid_610_624.orb", sep=" ", index_col=False, low_memory=False)
orbits = thor.readORBFile("../../analysis/ztf/MPCORB_20181106_ZTF_keplerian.orb", elementType="keplerian")

#observations = pd.read_csv("../analysis/ztf/known_object_observations_nid_610_624.txt", sep=" ", index_col=False, low_memory=False)
observations = pd.read_csv("../../analysis/ztf/observations_nid_610_624.txt", sep=" ", index_col=False, low_memory=False)

observations.sort_values(by="mjd", inplace=True)
#observations.drop(index=observations[observations["designation"].isna()].index, inplace=True)
observations.loc[observations["designation"].isna(), "designation"] = -1

In [7]:
allObjects_survey, summary_survey = thor.analyzeObservations(observations, columnMapping=columnMapping, minSamples=5, unknownIDs=[-1])

THOR: analyzeObservations
-------------------------
Analyzing observations...
Known object observations: 252836
Unknown object observations: 574710
False positive observations: 0
Percent known object observations (%): 30.553
Percent unknown object observations (%): 69.447
Percent false positive observations (%): 0.000
Unique known objects: 62307
Unique known objects with at least 5 detections: 21401

Total time in seconds: 0.4992187023162842
-------------------------



In [8]:
simulated_ephemeris["exp_mjd"] = simulated_ephemeris["mjd_utc"]

In [9]:
size = 15
ras = np.arange(0, 360 + size, size)
decs = np.arange(-90, 90 + size, size)

In [10]:
average_orbits_list = []

for ra_i, ra_f in zip(ras[:-1], ras[1:]):
    for dec_i, dec_f in zip(decs[:-1], decs[1:]):
        # See if  there are any observations in the patch
        observations_in_patch = observations[(observations[columnMapping["RA_deg"]] >= ra_i) 
                                             & (observations[columnMapping["RA_deg"]] < ra_f)
                                             & (observations[columnMapping["Dec_deg"]] < dec_f)
                                             & (observations[columnMapping["Dec_deg"]] >= dec_i)]
        if len(observations_in_patch) > 0:
            # Find time of first set of observations in patch (need to propagate orbit to that time)
            exp_mjd = observations_in_patch["exp_mjd"].min()
            
            average_orbits_hun1_patch = thor.findAverageOrbits(simulated_ephemeris[(simulated_ephemeris["RA_deg"] >= ra_i) 
                                                                                  & (simulated_ephemeris["RA_deg"] < ra_f)
                                                                                  & (simulated_ephemeris["Dec_deg"] < dec_f)
                                                                                  & (simulated_ephemeris["Dec_deg"] >= dec_i)],
                                                        orbits[(orbits["a_au"] < 2.06) & (orbits["a_au"] >= 1.7) & (orbits["e"] <= 0.1)],
                                                        elementType="keplerian",
                                                        dValues=[1.7, 2.06],
                                                        verbose=False)
            average_orbits_hun2_patch = thor.findAverageOrbits(simulated_ephemeris[(simulated_ephemeris["RA_deg"] >= ra_i) 
                                                                                  & (simulated_ephemeris["RA_deg"] < ra_f)
                                                                                  & (simulated_ephemeris["Dec_deg"] < dec_f)
                                                                                  & (simulated_ephemeris["Dec_deg"] >= dec_i)],
                                                         orbits[(orbits["a_au"] < 2.06) & (orbits["a_au"] >= 1.7) & (orbits["e"] > 0.1) & (orbits["e"] <= 0.2)],
                                                         elementType="keplerian",
                                                         dValues=[1.7, 2.06],
                                                         verbose=False)
            average_orbits_hun3_patch = thor.findAverageOrbits(simulated_ephemeris[(simulated_ephemeris["RA_deg"] >= ra_i) 
                                                                                  & (simulated_ephemeris["RA_deg"] < ra_f)
                                                                                  & (simulated_ephemeris["Dec_deg"] < dec_f)
                                                                                  & (simulated_ephemeris["Dec_deg"] >= dec_i)],
                                                         orbits[(orbits["a_au"] < 2.06) & (orbits["a_au"] >= 1.7) & (orbits["e"] > 0.2) & (orbits["e"] <= 0.4) ],
                                                         elementType="keplerian",
                                                         dValues=[1.7, 2.06],
                                                         verbose=False)

            average_orbits_patch = thor.findAverageOrbits(simulated_ephemeris[(simulated_ephemeris["RA_deg"] >= ra_i) 
                                                                                  & (simulated_ephemeris["RA_deg"] < ra_f)
                                                                                  & (simulated_ephemeris["Dec_deg"] < dec_f)
                                                                                  & (simulated_ephemeris["Dec_deg"] >= dec_i)],
                                                    orbits,
                                                    elementType="keplerian",
                                                    dValues=[2.06, 2.5, 2.82, 2.95, 3.27, 5.0],
                                                    verbose=False)
            average_orbits_patch = pd.concat([average_orbits_hun1_patch, average_orbits_hun2_patch, average_orbits_hun3_patch, average_orbits_patch])
            average_orbits_patch["exp_mjd_start"] = [exp_mjd for i in range(0, len(average_orbits_patch))]
            average_orbits_list.append(average_orbits_patch)
                                        
average_orbits = pd.concat(average_orbits_list)

average_orbits.sort_values(by=["a_au"], inplace=True)
average_orbits["orbit_id"] = np.arange(1, len(average_orbits) + 1)
average_orbits.reset_index(inplace=True, drop=True)

orbits["i_rad"] = np.radians(orbits["i_deg"])
orbits["ascNode_rad"] = np.radians(orbits["ascNode_deg"])
orbits["argPeri_rad"] = np.radians(orbits["argPeri_deg"])
orbits["meanAnom_rad"] = np.radians(orbits["meanAnom_deg"])

average_orbits = average_orbits.drop(index=average_orbits[average_orbits["designation"].isna()].index)

In [11]:
for orbit_id in average_orbits["orbit_id"].values:
    orbit = average_orbits[average_orbits["orbit_id"] == orbit_id]
    
    new_epoch = average_orbits[average_orbits["orbit_id"] == orbit_id]["exp_mjd_start"].values[0]
    
    orbit_from_cat = orbits[orbits["designation"] == orbit["designation"].values[0]]
    current_epoch = orbit_from_cat["epoch_TT_mjd"]
    
    eph = thor.propagateTestParticle(orbit_from_cat[["a_au", "e", "i_rad", "ascNode_rad", "argPeri_rad", "meanAnom_rad"]].values[0], current_epoch, [new_epoch], observatoryCode="I41", mjdScale="TT", elementType="keplerian")
    
    average_orbits.loc[average_orbits["orbit_id"] == orbit_id, "r_au"] = eph["r_au"].values[0]
    average_orbits.loc[average_orbits["orbit_id"] == orbit_id, "RA_deg"] = eph["RA_deg"].values[0]
    average_orbits.loc[average_orbits["orbit_id"] == orbit_id, "Dec_deg"] = eph["Dec_deg"].values[0]
    average_orbits.loc[average_orbits["orbit_id"] == orbit_id, "exp_mjd"] = eph["mjd"].values[0]
    average_orbits.loc[average_orbits["orbit_id"] == orbit_id, "HEclObj_dX/dt_au_p_day"] = eph["HEclObj_dX/dt_au_p_day"].values[0]
    average_orbits.loc[average_orbits["orbit_id"] == orbit_id, "HEclObj_dY/dt_au_p_day"] = eph["HEclObj_dY/dt_au_p_day"].values[0]
    average_orbits.loc[average_orbits["orbit_id"] == orbit_id, "HEclObj_dZ/dt_au_p_day"] = eph["HEclObj_dZ/dt_au_p_day"].values[0]

In [ ]:
average_orbits_in_observations = observations[observations["designation"].isin(average_orbits["designation"].values)].copy()
average_orbits_in_observations.drop_duplicates(subset=["designation"], keep="first", inplace=True)

In [12]:
average_orbits.head()

,orbit_id,r_au,HEclObj_dX/dt_au_p_day,HEclObj_dY/dt_au_p_day,HEclObj_dZ/dt_au_p_day,exp_mjd,RA_deg,Dec_deg,a_au,i_deg,e,designation,exp_mjd_start
0,1,1.599939,-0.000169,0.013336,0.004248,58364.356447,14.014296,-13.026011,1.700746,18.05810,0.208066,K17B91X,58364.356447
1,2,1.769631,0.003631,0.012197,-0.000025,58364.146458,287.056841,36.080929,1.715694,33.82898,0.277999,K17A03W,58364.146458
2,3,1.932523,0.011467,0.000194,0.001503,58364.134780,237.629370,16.500590,1.715840,36.59919,0.131734,K12TC4H,58364.134780
3,4,1.396965,0.000049,0.015809,0.001393,58364.184583,309.458975,43.034789,1.722594,23.96368,0.357560,K10A03C,58364.184583
4,5,1.297224,-0.012204,0.011242,0.003096,58364.509225,93.516653,0.643928,1.727334,23.49679,0.280285,U2800,58364.509225


In [13]:
average_orbits = average_orbits.rename(columns={"RA_deg": columnMapping["RA_deg"], "Dec_deg": columnMapping["Dec_deg"]})

In [19]:
! rm -rf ../../analysis/ztf/run_14/

In [15]:
observations = pd.merge(observations, simulated_ephemeris[["designation", "r_au"]], 
                        left_on="designation", 
                        right_on="designation", 
                        how="left",
                        suffixes=('', '_y'))

In [ ]:
allObjects_survey, summary_survey, summary_orbits = thor.runTHOR(observations, 
                                                                average_orbits,
                                                                knownOrbits=orbits,
                                                                runDir="/epyc/projects/thor/thor/analysis/ztf/run_14/", 
                                                                cellArea=500,
                                                                cellShape="circle",
                                                                threads=40,
                                                                vxBins=300,
                                                                vyBins=300,
                                                                eps=0.005,
                                                                minSamples=5,
                                                                unknownIDs=[-1],
                                                                columnMapping=columnMapping)

THOR: runTHOR
-------------------------
Running THOR with 764 orbits...

THOR: analyzeObservations
-------------------------
Analyzing observations...
Known object observations: 252836
Unknown object observations: 574710
False positive observations: 0
Percent known object observations (%): 30.553
Percent unknown object observations (%): 69.447
Percent false positive observations (%): 0.000
Unique known objects: 62307
Unique known objects with at least 5 detections: 21401

Total time in seconds: 0.5730459690093994
-------------------------

THOR: runTHOR
-------------------------
Running orbit 1...

THOR: analyzeProjections
-------------------------
Analyzing projections...
Known object observations: 23872
Unknown object observations: 6444
False positive observations: 0
Percent known object observations (%): 78.744
Percent unknown object observations (%): 21.256
Percent false positive observations (%): 0.000
Unique known objects: 5469
Unique known objects with at least 5 detections: 242

../../thor/analysis.py:572: RuntimeWarning:

divide by zero encountered in true_divide

../../thor/analysis.py:573: RuntimeWarning:

divide by zero encountered in true_divide

../../thor/analysis.py:574: RuntimeWarning:

divide by zero encountered in true_divide

/epyc/projects/thor/envs/thor/lib/python3.6/site-packages/numpy/core/_methods.py:117: RuntimeWarning:

invalid value encountered in subtract

../../thor/analysis.py:575: RuntimeWarning:

divide by zero encountered in true_divide



Known object observations: 2875
Unknown object observations: 2090
False positive observations: 0
Percent known object observations (%): 57.905
Percent unknown object observations (%): 42.095
Percent false positive observations (%): 0.000
Unique known objects: 975
Unique known objects with at least 5 detections: 203

Total time in seconds: 4.346774578094482
-------------------------

THOR: clusterAndLink
-------------------------
Running velocity space clustering...
X velocity range: [-0.1, 0.1]
X velocity bins: 300
Y velocity range: [-0.1, 0.1]
Y velocity bins: 300
User defined x velocity values: False
User defined y velocity values: False
Velocity grid size: 90000
Max sample distance: 0.005
Minimum samples: 5
Using 40 threads...
Done. Completed in 79.82014894485474 seconds.

Restructuring clusters...
Done. Completed in 0.4073822498321533 seconds.

Found 2763 clusters.
Total time in seconds: 80.22838878631592
-------------------------

THOR: analyzeClusters
-------------------------
An